In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.linear_model import LinearRegression
import seaborn as sns

# Przygotowanie danych
data = pd.DataFrame({
    'group': range(1, 11),
    'total': [1000, 1500, 900, 800, 1000, 800, 400, 200, 100, 400],
    'age_range': ['19-22', '23-26', '27-30', '31-34', '35-38', '39-42', '43-46', '47-50', '51-54', '55-58'],
    'work_experience': [1, 3, 5, 7, 9, 20, 20, 25, 28, 27],
    'found_job': [100, 160, 110, 130, 180, 200, 110, 60, 33, 13]
})

# Obliczenie średniego wieku i prawdopodobieństwa sukcesu
data['age_mean'] = data['age_range'].apply(lambda x: np.mean([int(i) for i in x.split('-')]))
data['success_prob'] = data['found_job'] / data['total']

# Transformacja logitowa
data['logit'] = np.log(data['success_prob'] / (1 - data['success_prob']))

# Utworzenie macierzy X dla obu zmiennych niezależnych
X = data[['age_mean', 'work_experience']]

# Model liniowy na transformowanych danych logitowych
logit_model = LinearRegression()
logit_model.fit(X, data['logit'])

# Zwykły model liniowy na oryginalnych prawdopodobieństwach
linear_model = LinearRegression()
linear_model.fit(X, data['success_prob'])

# Przygotowanie danych do wizualizacji
X_plot = np.linspace(20, 60, 100)
exp_plot = np.linspace(1, 30, 100)
X_grid, exp_grid = np.meshgrid(X_plot, exp_plot)
X_pred = np.column_stack([X_grid.ravel(), exp_grid.ravel()])

# Przewidywania modeli
logit_pred = expit(logit_model.predict(X_pred))
linear_pred = linear_model.predict(X_pred)

# Wizualizacja
plt.figure(figsize=(15, 10))

# 1. Krzywa logistyczna vs liniowa dla wieku (przy średnim stażu)
plt.subplot(221)
mean_exp = data['work_experience'].mean()
X_age = np.column_stack([X_plot, np.full_like(X_plot, mean_exp)])
plt.plot(X_plot, expit(logit_model.predict(X_age)), 'b-', label='Model logistyczny')
plt.plot(X_plot, linear_model.predict(X_age), 'r--', label='Model liniowy')
plt.scatter(data['age_mean'], data['success_prob'],
           s=data['total']/30, alpha=0.6,
           label='Dane rzeczywiste')
plt.xlabel('Wiek')
plt.ylabel('Prawdopodobieństwo znalezienia pracy')
plt.title('Porównanie modeli (przy średnim stażu pracy)')
plt.legend()

# 2. Transformacja logitowa
plt.subplot(222)
plt.scatter(data['age_mean'], data['logit'],
           s=data['total']/30, alpha=0.6,
           label='Dane transformowane')
plt.plot(X_plot, logit_model.predict(X_age), 'b-',
         label='Regresja liniowa na\ntransformowanych danych')
plt.xlabel('Wiek')
plt.ylabel('Logit prawdopodobieństwa')
plt.title('Transformacja logitowa danych')
plt.legend()

# 3. Porównanie przewidywań
plt.subplot(223)
plt.scatter(data['success_prob'],
           expit(logit_model.predict(X)),
           label='Model logistyczny', alpha=0.6)
plt.scatter(data['success_prob'],
           linear_model.predict(X),
           label='Model liniowy', alpha=0.6)
plt.plot([0, 0.3], [0, 0.3], 'k--')
plt.xlabel('Rzeczywiste prawdopodobieństwo')
plt.ylabel('Przewidywane prawdopodobieństwo')
plt.title('Porównanie przewidywań modeli')
plt.legend()

# 4. Mapa ciepła dla modelu logistycznego
plt.subplot(224)
plt.contourf(X_grid, exp_grid, logit_pred.reshape(X_grid.shape),
            levels=20, cmap='viridis')
plt.colorbar(label='Prawdopodobieństwo znalezienia pracy')
plt.scatter(data['age_mean'], data['work_experience'],
           c=data['success_prob'], cmap='viridis',
           s=100, edgecolor='white')
plt.xlabel('Wiek')
plt.ylabel('Staż pracy')
plt.title('Model logistyczny: wpływ wieku i stażu')

plt.tight_layout()
plt.show()

# Wydruk współczynników modeli
print("\nWspółczynniki modelu logistycznego:")
print(f"Wyraz wolny: {logit_model.intercept_:.4f}")
print(f"Wiek: {logit_model.coef_[0]:.4f}")
print(f"Staż pracy: {logit_model.coef_[1]:.4f}")

print("\nWspółczynniki modelu liniowego:")
print(f"Wyraz wolny: {linear_model.intercept_:.4f}")
print(f"Wiek: {linear_model.coef_[0]:.4f}")
print(f"Staż pracy: {linear_model.coef_[1]:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, LogisticRegression
from scipy.special import expit
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Przygotowanie danych
data = pd.DataFrame({
    'group': range(1, 11),
    'total': [1000, 1500, 900, 800, 1000, 800, 400, 200, 100, 400],
    'age_range': ['19-22', '23-26', '27-30', '31-34', '35-38', '39-42', '43-46', '47-50', '51-54', '55-58'],
    'work_experience': [1, 3, 5, 7, 9, 20, 20, 25, 28, 27],
    'found_job': [100, 160, 110, 130, 180, 200, 110, 60, 33, 13]
})

# Obliczenie średniego wieku i prawdopodobieństwa sukcesu
data['age_mean'] = data['age_range'].apply(lambda x: np.mean([int(i) for i in x.split('-')]))
data['success_prob'] = data['found_job'] / data['total']
data['logit'] = np.log(data['success_prob'] / (1 - data['success_prob']))

# Dodanie potencjalnych nowych zmiennych
data['age_squared'] = data['age_mean'] ** 2
data['exp_squared'] = data['work_experience'] ** 2
data['age_exp_interaction'] = data['age_mean'] * data['work_experience']

# Wybór zmiennych do analizy korelacji
features = ['age_mean', 'work_experience', 'age_squared', 'exp_squared', 'age_exp_interaction']
target = 'success_prob'

# Macierz korelacji
correlation_matrix = data[features + [target]].corr()

# Analiza VIF
X = data[features]
vif_data = pd.DataFrame()
vif_data["Variable"] = features
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Wizualizacja
plt.figure(figsize=(15, 12))

# 1. Mapa ciepła korelacji
plt.subplot(221)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Macierz korelacji')

# 2. Scatter plot dla wieku
plt.subplot(222)
plt.scatter(data['age_mean'], data['success_prob'])
plt.xlabel('Wiek')
plt.ylabel('Prawdopodobieństwo sukcesu')
plt.title('Wiek vs Prawdopodobieństwo')

# 3. Scatter plot dla doświadczenia
plt.subplot(223)
plt.scatter(data['work_experience'], data['success_prob'])
plt.xlabel('Staż pracy')
plt.ylabel('Prawdopodobieństwo sukcesu')
plt.title('Staż vs Prawdopodobieństwo')

# 4. VIF jako wykres słupkowy
plt.subplot(224)
plt.bar(vif_data['Variable'], vif_data['VIF'])
plt.xticks(rotation=45)
plt.ylabel('VIF')
plt.title('Współczynniki VIF')

plt.tight_layout()
plt.show()

# Modele na podstawowych zmiennych
X_base = data[['age_mean', 'work_experience']]

# Model liniowy
linear_model = LinearRegression()
linear_model.fit(X_base, data['success_prob'])

# Model logistyczny (na transformowanych danych)
logit_model = LinearRegression()
logit_model.fit(X_base, data['logit'])

# Wizualizacja modeli
plt.figure(figsize=(15, 5))

# 1. Porównanie modeli przy średnim stażu
mean_exp = data['work_experience'].mean()
X_age = np.linspace(20, 60, 100)
X_pred = np.column_stack([X_age, np.full_like(X_age, mean_exp)])

plt.subplot(131)
plt.scatter(data['age_mean'], data['success_prob'], label='Dane rzeczywiste')
plt.plot(X_age, linear_model.predict(X_pred), 'r--', label='Model liniowy')
plt.plot(X_age, expit(logit_model.predict(X_pred)), 'b-', label='Model logistyczny')
plt.xlabel('Wiek')
plt.ylabel('Prawdopodobieństwo')
plt.title('Porównanie modeli (przy średnim stażu)')
plt.legend()

# 2. Transformacja logitowa
plt.subplot(132)
plt.scatter(data['age_mean'], data['logit'], label='Dane transformowane')
plt.plot(X_age, logit_model.predict(X_pred), 'b-', label='Regresja liniowa\nna transformowanych danych')
plt.xlabel('Wiek')
plt.ylabel('Logit prawdopodobieństwa')
plt.title('Transformacja logitowa')
plt.legend()

# 3. Mapa ciepła dla modelu logistycznego
plt.subplot(133)
age_grid, exp_grid = np.meshgrid(np.linspace(20, 60, 50), np.linspace(1, 30, 50))
X_grid = np.column_stack([age_grid.ravel(), exp_grid.ravel()])
Z = expit(logit_model.predict(X_grid)).reshape(age_grid.shape)

plt.contourf(age_grid, exp_grid, Z, levels=20, cmap='viridis')
plt.colorbar(label='Prawdopodobieństwo')
plt.scatter(data['age_mean'], data['work_experience'], c=data['success_prob'],
           cmap='viridis', s=100, edgecolor='white')
plt.xlabel('Wiek')
plt.ylabel('Staż pracy')
plt.title('Model logistyczny: wpływ wieku i stażu')

plt.tight_layout()
plt.show()

# Wyniki analizy
print("\nWspółczynniki modelu liniowego:")
print(f"Wyraz wolny: {linear_model.intercept_:.4f}")
print(f"Wiek: {linear_model.coef_[0]:.4f}")
print(f"Staż: {linear_model.coef_[1]:.4f}")

print("\nWspółczynniki modelu logistycznego (na skali logit):")
print(f"Wyraz wolny: {logit_model.intercept_:.4f}")
print(f"Wiek: {logit_model.coef_[0]:.4f}")
print(f"Staż: {logit_model.coef_[1]:.4f}")

print("\nWspółczynniki VIF:")
print(vif_data)

print("\nKorelacje z prawdopodobieństwem sukcesu:")
for feature in features:
    correlation = correlation_matrix.loc[feature, target]
    print(f"{feature}: {correlation:.3f}")